# Quickstart
- [Tutorial link](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html)

PyTorch has two primitives to work with data 
- `torch.utils.data.Dataset` - stores the samples and their corresponding labels
- `torch.utils.data.DataLoader` - wraps an iterable around the dataset

## Working with Data

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [4]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100.0%


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



12.6%

100.0%


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100.0%

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [6]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## Creating Models

To define a neural network, we create a class that inherits from `nn.Module`

We define the layers of the network in the `__init__` function and specify how data will pass through the network in the
`forward` function.

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Optimizing the Model Parameters

To train a model, we need a loss function and an optimizer.

In [12]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In a single training loop, the model makes predictions on the training dataset (fed to in in batches), 
and backpropagates the prediction error to adjust the model's parameters

In [19]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

We also check the model's performance against the test dataset to ensure it is learning.

In [15]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {100*correct:>0.1f}%, Avg loss: {test_loss:>8f} \n")

The training process is conducted over sseveral iterations (epochs).
During each epoch, the model learns parameters to make better predictions.
We print the model's accuracy and loss at each epoch - we'd like to see the accuracy
increase and loss decrease with every epoch.

In [20]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n" + "-"*16)
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
----------------
loss: 2.310920 [    0/60000]
loss: 2.292488 [ 6400/60000]
loss: 2.275543 [12800/60000]
loss: 2.267533 [19200/60000]
loss: 2.242677 [25600/60000]
loss: 2.211573 [32000/60000]
loss: 2.213006 [38400/60000]
loss: 2.181560 [44800/60000]
loss: 2.192401 [51200/60000]
loss: 2.136940 [57600/60000]
Test Error: 
 Accuracy: 47.5%, Avg loss: 2.140749 

Epoch 2
----------------
loss: 2.159411 [    0/60000]
loss: 2.142035 [ 6400/60000]
loss: 2.081956 [12800/60000]
loss: 2.098908 [19200/60000]
loss: 2.041536 [25600/60000]
loss: 1.975562 [32000/60000]
loss: 1.994197 [38400/60000]
loss: 1.915014 [44800/60000]
loss: 1.934292 [51200/60000]
loss: 1.830224 [57600/60000]
Test Error: 
 Accuracy: 52.8%, Avg loss: 1.842322 

Epoch 3
----------------
loss: 1.885419 [    0/60000]
loss: 1.844761 [ 6400/60000]
loss: 1.726050 [12800/60000]
loss: 1.766265 [19200/60000]
loss: 1.664948 [25600/60000]
loss: 1.606754 [32000/60000]
loss: 1.624752 [38400/60000]
loss: 1.531796 [44800/60000]
loss: 1.5

## Saving models
A common way to save  model is to serialize the internal state dictionary (containing the model parameters)

In [21]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## Loading Models
The process for loading a model includes re-creating the model and loading the state dictionary into it.

In [25]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

The model can now be used to make predictions

In [27]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "ankle boot"
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x.cuda()) # make sure it's cuda here, otherwise it's an error
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f"Predicted: '{predicted}', Actual: '{actual}'")

Predicted: 'ankle boot', Actual: 'ankle boot'
